# Texture Classification via SVM

<h4>Project utilizing Support Vector Machine models to recognize and classify 2D texture images.</h4>
<h4>Dataset comprised a repository of hand-collected images prepared for training the model.</h4>

## Problem: Texture Tiling in 3D Asset Creation

2D Texture Tiling occurs due to repeat images (openGL tiling) that do not fit models exactly.
*Image of poorly tiled Blender Wall etc., credit Blender Guru and link his video here.
### Preparation for Future Neural Network Procedural Synthetic Texture Genesis (Texture Synthesis)

## Exploration of the Image Collection

Data came from my own walking about and taking photos like a goober (we'll call this phogrammetric loitery)
- Size is 'x' classes of texture and 'n' images of each
- Perform data analysis and investigate the data, visualize relationship between classes
- Data cleaning
### Some Stuff about me taking photos outside

## Preparing the Data For Processing

talk about
Apply relevant data transformation (i.e., normalization or encoding etc) for the variables
- If the dataset is impbalanced, apply techniques such as oversampling and undersampling, SMOTE (synthetic minority over sampling technique) etc

## Model Building

### Jump into the conversation with a number of papers
Justify selection of algorithm, and train multiple models (min of 2) and compare their performances
Tune some hyperparameters like grid serach, random search, bayesian optimization.

## Evaluation

### Maybe some stuff in Blender or Substance
evaluate model performance (accuracy, precision, recall)
perform cross-validation and report average performance (across k-fold cross validation or some such)

## Conclusions and Next Steps

discuss insights drawn from the model, how does it solve the problem i set out on? limitations to the problem (such as not creating the textures yet)